In [ ]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import platform
import sys
sys.path.append('/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/'
                if platform.uname().processor == 'x86_64' 
                else '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import argparse
import os
import re
import json
import tqdm
import glob
import torch
import random
import evaluate
import torch
from eval.utils import (
    load_hf_lm_and_tokenizer,
    generate_completions,
    query_openai_chat_model,
    dynamic_import_function,
)
from transformers import GPT2LMHeadModel

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument("--data_dir", type=str, default="data/bbh")
parser.add_argument("--save_dir", type=str, default="results/bbh")
parser.add_argument("--model_name_or_path", type=str, default=None, help="if specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="if specified, we will load the tokenizer from here.")
parser.add_argument("--use_slow_tokenizer", action="store_true", help="If given, we will use the slow tokenizer.")
parser.add_argument("--openai_engine", type=str, default=None, help="if specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--no_cot", action="store_true", help="if specified, chain of thoughts will be removed from the prompts.")
parser.add_argument("--max_num_examples_per_task", type=int, default=None, help="maximum number of examples to evaluate per task.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, the prompt will be encoded as a chat format with the roles in prompt.")
parser.add_argument("--chat_formatting_function", type=str, default="eval.templates.create_prompt_with_tulu_chat_format", help="The function to use to create the chat format. This function will be dynamically imported. Please see examples in `eval/templates.py`.")
parser.add_argument("--max_new_tokens", type=int, default=256)
parser.add_argument("--n_shot", type=int, default=3)


model_name_or_path = '../results/baselines/gpt2-large'
# model_name_or_path = '../results/baselines/gpt2-medium'
# model_name_or_path = '../results/gpt2-Large_human_mix'
# model_name_or_path = '../results/huggyllama:llama-7b_human_mix-trainer_savebystep/checkpoint-200'
model_name_or_path = '../results/baselines/huggyllama/llama-7b/'

cmd = f"""
    --data_dir ../data/eval/bbh/ \
    --save_dir {model_name_or_path}/eval/bbh/ \
    --max_num_examples_per_task 40 \
    --model_name_or_path {model_name_or_path} \
    --eval_batch_size 10
"""

args = jpt_parse_args(parser, cmd)


# model_name_or_path and openai_engine cannot be both None or both not None.
assert (args.model_name_or_path is None) != (args.openai_engine is None), "Either model_name_or_path or openai_engine should be specified."

args

In [ ]:
args.exact_match = evaluate.load("exact_match", experiment_id=args.model_name_or_path)


In [ ]:
random.seed(42)

data_size = 0
all_tasks = {}
task_files = glob.glob(os.path.join(args.data_dir, "bbh", "*.json"))
for task_file in tqdm.tqdm(task_files, desc="Loading tasks"):
    with open(task_file, "r") as f:
        task_name = os.path.basename(task_file).split(".")[0]
        all_tasks[task_name] = json.load(f)["examples"]
        print(f'{task_name}\t{len(all_tasks[task_name])}')
        data_size += len(all_tasks[task_name])
        if args.max_num_examples_per_task:
            all_tasks[task_name] = random.sample(all_tasks[task_name], args.max_num_examples_per_task)
data_size

In [ ]:
dynamic_import_function(args.chat_formatting_function)

In [ ]:

all_prompts = {}
cot_prompt_files = glob.glob(os.path.join(args.data_dir, "cot-prompts", "*.txt"))
for cot_prompt_file in tqdm.tqdm(cot_prompt_files, desc="Loading prompts"):
    with open(cot_prompt_file, "r") as f:
        task_name = os.path.basename(cot_prompt_file).split(".")[0]
        task_prompt = "".join(f.readlines()[2:])

        prompt_fields = task_prompt.split("\n\n")
        # `new_prompt_fields[0]`: sub-task instruction/description
        # `new_prompt_fields[1:]`: demonstrations
        new_prompt_fields = []
        for prompt_field in prompt_fields:
            if prompt_field.startswith("Q:"):
                assert "So the answer is" in prompt_field, f"`So the answer is` not found in prompt field of {task_name}.txt."
                assert "\nA:" in prompt_field, "`\nA:` not found in prompt field."
                question, answer = prompt_field.split("\nA:")
                question, answer = question.strip(), answer.strip()
                if args.no_cot:
                    answer = answer.split("So the answer is")[-1].strip()
                new_prompt_fields.append(question+'\nA: '+answer)
            else:
                new_prompt_fields.append(prompt_field)
        # prompt instruction span two lines! concat them.
        if task_name == 'snarks':
            new_prompt_fields = ['\n\n'.join(new_prompt_fields[:2])]+new_prompt_fields[2:]
        assert(len(new_prompt_fields) == 4)
        all_prompts[task_name] = new_prompt_fields

# ## previous impl. modified to enable using <=3 in-context examples when prompt is too long.
# 
#     with open(cot_prompt_file, "r") as f:
#         task_name = os.path.basename(cot_prompt_file).split(".")[0]
#         task_prompt = "".join(f.readlines()[2:])
#         if args.no_cot:
#             prompt_fields = task_prompt.split("\n\n")
#             new_prompt_fields = []
#             for prompt_field in prompt_fields:
#                 if prompt_field.startswith("Q:"):
#                     assert "So the answer is" in prompt_field, f"`So the answer is` not found in prompt field of {task_name}.txt."
#                     assert "\nA:" in prompt_field, "`\nA:` not found in prompt field."
#                     answer = prompt_field.split("So the answer is")[-1].strip()
#                     question = prompt_field.split("\nA:")[0].strip()
#                     new_prompt_fields.append(question + "\nA: " + answer)
#                 else:
#                     new_prompt_fields.append(prompt_field)
#             task_prompt = "\n\n".join(new_prompt_fields)
#         all_prompts[task_name] = task_prompt

assert set(all_tasks.keys()) == set(all_prompts.keys()), "task names in task data and task prompts are not the same."

In [ ]:
# task_prompt = all_prompts[task_name]
# examples = all_tasks[task_name]

# def get_task_prompt(n_shot):
#     assert(0 <= n_shot <= 3)
#     return "\n\n".join(task_prompt[:n_shot+1]).strip()

# prompts = []
# for example in examples:
#     for n_shot in list(range(args.n_shot+1)[::-1]):
#         task_prompt_concat = get_task_prompt(n_shot)
#         if args.use_chat_format:
#             prompt = "<|user|>\n" + task_prompt_concat + "\n\nQ: " + example["input"] + "\n<|assistant|>\nA:"
#         else:
#             prompt = task_prompt_concat + "\n\nQ: " + example["input"] + "\nA:"
#         tokenized_prompt_len = len(tokenizer(prompt, add_special_tokens=False)['input_ids'])
#         if tokenized_prompt_len < max_input_seq_len:
#             break
#     if n_shot != args.n_shot:
#         print(f'n_shot: {args.n_shot} -> {n_shot}')
#     prompts.append(prompt)

# max_input_seq_len

In [ ]:
# # print(prompt_field)

# question, answer = prompt_field.split("\nA:")
# question, answer = question.strip(), answer.strip()
# if args.no_cot:
#     answer = answer.split("So the answer is")[-1].strip()
# answer =  "A: " + answer

In [ ]:

os.makedirs(args.save_dir, exist_ok=True)
os.makedirs(os.path.join(args.save_dir, "predictions"), exist_ok=True)


if args.model_name_or_path:
    print("Loading model and tokenizer...")
    model, tokenizer = load_hf_lm_and_tokenizer(
        model_name_or_path=args.model_name_or_path, 
        tokenizer_name_or_path=args.tokenizer_name_or_path, 
        load_in_8bit=args.load_in_8bit, 
        device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
        gptq_model=args.gptq,
        use_fast_tokenizer=not args.use_slow_tokenizer,
    )
    
# wpq: for gpt-2 model, need to enforce `max_length` constraints to avoid `position_id` index errors.
if isinstance(model, GPT2LMHeadModel):
    max_input_seq_len = model.config.max_position_embeddings - args.max_new_tokens
else:
    max_input_seq_len = 2048 - args.max_new_tokens

    
model.device, model.dtype

In [ ]:
task_name = next(iter(all_tasks.keys()))
task_examples = all_tasks[task_name]
prompt = all_prompts[task_name]
print(prompt)
print(len(prompt))
print()
print(task_examples[0])

In [ ]:
# task_perf = eval_hf_model(
#     args, 
#     model, 
#     tokenizer, 
#     task_examples, 
#     prompt, 
#     save_path=os.path.join(args.save_dir, "predictions", f"{task_name}.jsonl")
# )

# performance[task_name] = task_perf
# print(f"Task {task_name} - EM: {task_perf}")

examples = task_examples
task_prompt = prompt
save_path=os.path.join(args.save_dir, "predictions", f"{task_name}.jsonl")


In [ ]:
targets = [example["target"] for example in examples]
if save_path:
    fout = open(save_path, "w")
save_path

In [ ]:

## wpq: for k=3 shots, reduce number of demonstrations if the prompt is too long.
prompts = []
chat_formatting_function = dynamic_import_function(args.chat_formatting_function) if args.use_chat_format else None
for example in examples:
    for n_shot in list(range(args.n_shot+1)[::-1]):
        task_prompt_concat = get_task_prompt(n_shot)
        prompt = task_prompt_concat + "\n\nQ: " + example["input"]
        if args.use_chat_format:
            messages = [{"role": "user", "content": prompt}]
            prompt = chat_formatting_function(messages, add_bos=False)
            if prompt[-1] in ["\n", " "]:
                prompt += "A:"
            else:
                prompt += " A:"
        else:
            prompt += "\nA:"
        tokenized_prompt_len = len(tokenizer(prompt, add_special_tokens=False)['input_ids'])
        if tokenized_prompt_len <= max_input_seq_len:
            break
    if n_shot != args.n_shot:
        print(f'n_shot: {args.n_shot} -> {n_shot}')
    prompts.append(prompt)

print(prompts[0])

In [ ]:
for s in [' ', '\n', '\n\n', 'A: (B)\n\n']:
    print(s, tokenizer.encode(s, add_special_tokens=False))

In [ ]:
l = []
for s in [[13], [13,13], [29871,13]]:
    l.append(tokenizer.decode(s, add_special_tokens=False))
l

In [ ]:
if args.no_cot:
    # get the last token because the tokenizer may add space tokens at the start.
    stop_id_sequences = tokenizer.encode("\n\n", add_special_tokens=False)
    stop_id_sequences = [stop_id_sequences[-2:]] if stop_id_sequences else None
else:
    # let's not use the stop sequence for cot now since it's too inefficient when the generation is long. 
    # instead, we'll do some post-processing to extract the answer.
    stop_sequnce = None

from transformers import GPT2LMHeadModel
if isinstance(model, GPT2LMHeadModel):
    # wpq: for gpt-2 model, need to enforce `max_length` constraints to avoid `position_id` index errors.
    generation_kwargs = {'max_length': model.config.max_position_embeddings} # 1024
else:
    # wpq: modify `max_new_tokens=512` to `128` for faster generation.
    # for non-cot multiple choice answers, e.g., ' (G).' requires just 5 tokens
    generation_kwargs = {'max_new_tokens': 10 if args.no_cot else 256}

outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts,
    batch_size=args.eval_batch_size if args.eval_batch_size else 1,
    stop_id_sequences=stop_id_sequences,
    **generation_kwargs,
)

In [ ]:


predictions = []
for example, output in zip(examples, outputs):
    example["raw_output"] = output

    # only keep the first part of the output - this is mainly for vanilla language models.
    output = output.strip().split("\n\n")[0].strip()

    # extract the first answer after `So the answer is` and before the next period.
    # if there is no such answer, we will just use the raw output.
    results = re.search(r"So the answer is (.*?)\.", output)
    if results:
        prediction = results.group(1).strip()
    else:
        prediction = output.strip()

    example["prediction"] = prediction
    predictions.append(prediction)
    if save_path:
        fout.write(json.dumps(example) + "\n")        

assert len(predictions) == len(targets), "number of predictions and targets are not the same."
task_perf = args.exact_match.compute(predictions=predictions, references=targets, ignore_case=True, ignore_punctuation=True)["exact_match"]
task_perf

In [ ]:
performance = {}
performance[task_name] = .1

In [ ]:

with open(os.path.join(args.save_dir, "metrics.json"), "w") as fout:
    performance["average_exact_match"] = sum(performance.values()) / len(performance)
    print(f"Average EM: {performance['average_exact_match']}")
    json.dump(performance, fout, indent=4)
